# Introduction
Ce code a pour objectif de générer des figures au format PNG à partir de données contenues dans des fichiers CSV.

Ces fichiers CSV sont situés dans le répertoire **ACQUISITION FORGEAGE**, tandis que les données de sortie, y compris les figures générées, sont enregistrées dans le répertoire **Output**.
Le fichier principal du projet est main.py, qui contient le code permettant de traiter les données et de générer les figures.

# Structure
- Ces fichiers CSV sont situés dans le répertoire **ACQUISITION FORGEAGE**, tandis que les données de sortie, y compris - les figures générées, sont enregistrées dans le répertoire **Output**.
- Le fichier principal du projet est main.py , qui contient le code clean  permettant de traiter les données et de générer les figures. 
- Le Jupyternotebook **main.ipynb** contenant un code explicative avec une description et des commentaires

# Décomposition
- Le code est  décomposé de deux parties : 
    - La première partie consiste à déterminer  l'axe du temps (**axe_temps**).
    - La deuxième partie parcout  les élements du fichier **ACQUISITION FORGEAGE**, et pour chaque élement, une figure est tracée et sauvegardée dans **Output/figures**
    

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
# Définir le chemin du répertoire contenant les fichiers de données
chemin_repertoire = "output"


# Définir le chemin du répertoire pour sauvegarder les figures
chemin_figures = os.path.join(chemin_repertoire, "figures")

# Vérifier si le répertoire "figures" existe, sinon le créer
if not os.path.exists(chemin_figures):
    os.makedirs(chemin_figures)

# Chemin du fichier CSV contenant les données
FICHIER_AXE_TEMPS = r"AXE_DU_TEMPS\ESSAI_1_3_6.csv"
FICHIER_REF_DATA2 = r"ACQUISITION FORGEAGE\ESSAI_114.csv"
# Charger les données du fichier CSV
df_axe_temps = pd.read_csv(FICHIER_AXE_TEMPS, delimiter=';')

# Sélectionner la dernière colonne "tps" comme axe du temps
axe_temps = df_axe_temps['tps']

# Enregistrer l'axe du temps dans un nouveau fichier CSV
axe_temps.to_csv(r'AXE_DU_TEMPS\AXE_TEMPS.csv', index=False)

fichiers_non_coherents = []

# Récupérer les colonnes de référence pour chaque type de fichier
colonnes_reference1 = pd.read_csv(FICHIER_AXE_TEMPS, delimiter=';', nrows=0).columns.tolist()
colonnes_reference2 = pd.read_csv(FICHIER_REF_DATA2, delimiter=';', nrows=0).columns.tolist()



In [2]:
# Définir les plages d'essais selon la séquence spécifiée 
plages_essais = [ (1,7),(11,17),(21,27),(31,37),(41, 47), (51, 57), (61, 67),
                 (71, 77), (81, 87), (91, 97), (101, 107), (111, 117), (121, 127)]

for plage in plages_essais:
    debut, fin = plage
    for essai in range(debut, fin + 1):
        if essai == 113:
            continue
        # Construire le chemin complet du fichier CSV de l'essai
        nom_fichier = os.path.join("ACQUISITION FORGEAGE", f"ESSAI_{str(essai).zfill(3)}.csv")
        print(nom_fichier)
        print(os.getcwd()) 
        if os.path.exists(nom_fichier):
            print('exist')
            try:
                # Charger les données à partir du fichier CSV de l'essai à partir de la deuxième ligne
                df_essai = pd.read_csv(nom_fichier, delimiter=";", skiprows=0)
                print(df_essai)

                # Trouver l'index de la dernière valeur non NaN dans les données
                dernier_index_non_nan = df_essai.apply(pd.Series.last_valid_index)

                # Trouver l'index le plus petit parmi les colonnes
                dernier_index = min(dernier_index_non_nan)

                # Tronquer l'axe des temps pour correspondre à la longueur de df_essai
                axe_temps_tronque = axe_temps[:dernier_index + 1]

                # Tronquer les données de chaque colonne à l'index de la dernière valeur non NaN
                df_essai_tronque = df_essai.iloc[:dernier_index + 1, :]

                # Extraction des données utiles
                df_essai = df_essai_tronque[:len(axe_temps_tronque)]

                # Conversion des données en float
                for col in df_essai.columns:
                    if df_essai[col].dtype == 'object':  # Vérifier si le type de données est une chaîne de caractères
                        df_essai[col] = df_essai[col].str.replace(',', '.').astype(float)  # Convertir les virgules en points décimaux et les colonnes en float
                    elif df_essai[col].dtype == 'float64' and df_essai[col].isnull().any():  # Vérifier si le type de données est float et s'il y a des NaN
                        df_essai[col] = df_essai[col].astype(float)  # Convertir les colonnes en float

                for col in df_essai.columns:
                    # Tracer chaque courbe avec le nom de colonne correspondant
                    plt.plot(axe_temps_tronque, df_essai[col], label=col)

                # Ajouter des titres et des légendes
                plt.title(f"Essai {essai}")
                plt.xlabel("Temps")  # Définir le nom de l'axe x
                plt.ylabel("Valeurs")
                plt.legend(loc='upper right')  # Ajouter la légende avec les noms de colonnes

                # Afficher et sauvegarder le graphique dans le dossier "figures"
                plt.grid(True)
                nom_figure = os.path.join(chemin_figures, f"ESSAI_{str(essai).zfill(3)}.png")
                plt.savefig(nom_figure)
                plt.close()
            except FileNotFoundError:
                print("FileNotFoundError")
                continue  # Passer à l'essai suivant s'il n'y a pas de fichier pour cet essai

print("Les graphiques ont été générés avec succès.")



ACQUISITION FORGEAGE\ESSAI_001.csv
c:\Users\linaz\OneDrive\Bureau\StageOpenData-main
exist
      SPR400_DPL SPR400_Effort KistlerKBeam_8396A050CTTA00_5506138_X  \
0              0  -0,150456002                          -0,830273145   
1              0  -1,969825406                          -0,832289738   
2              0   2,430815851                          -0,812527124   
3              0   2,003887897                          -0,829063189   
4              0   1,478438129                          -0,858908769   
...          ...           ...                                   ...   
18395        NaN           NaN                                   NaN   
18396        NaN           NaN                                   NaN   
18397        NaN           NaN                                   NaN   
18398        NaN           NaN                                   NaN   
18399        NaN           NaN                                   NaN   

      KistlerKBeam_8396A050CTTA00_5506138_Y 

KeyboardInterrupt: 